In [1]:
import pandas as pd
import sys
sys.path.append('../')

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-04-27/departures.csv')

In [3]:
from datasets.crsp_data import CRSP

In [5]:
crsp = CRSP()

DOWNLOADING RAW FILE
DOWNLOADING RAW FILE


Downloading...
From (original): https://drive.google.com/uc?id=15E7hEZdUf9nVVzlZokPVkanglF4j_Vni
From (redirected): https://drive.google.com/uc?id=15E7hEZdUf9nVVzlZokPVkanglF4j_Vni&confirm=t&uuid=1ba83269-c97b-4444-99cf-08fd56957066
To: /Users/sethjpete/data/crsp_monthly_raw.csv
100%|██████████| 1.46G/1.46G [01:28<00:00, 16.6MB/s]


CLEANING RAW FILE
CLEANING RAW FILE


/Users/sethjpete/Documents/Research/ceo_deaths/datasets/crsp_data.py:29: DtypeWarning: Columns (5,6,9,18,19,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(RAW_FILE_PATH)


LOADING CLEAN FILE


In [6]:
crsp.df

,permno,date,cusip,shrcd,exchcd,ticker,shrout,vol,prc,ret
0,10000,1986-02-28,68391610,10.0,3.0,OMFGA,3680.0,828.0,3.25000,-0.257143
1,10000,1986-03-31,68391610,10.0,3.0,OMFGA,3680.0,1078.0,4.43750,0.365385
2,10000,1986-04-30,68391610,10.0,3.0,OMFGA,3793.0,957.0,4.00000,-0.098592
3,10000,1986-05-30,68391610,10.0,3.0,OMFGA,3793.0,1074.0,3.10938,-0.222656
4,10000,1986-06-30,68391610,10.0,3.0,OMFGA,3793.0,1069.0,3.09375,-0.005025
...,...,...,...,...,...,...,...,...,...,...
3785681,93436,2023-08-31,88160R10,11.0,3.0,TSLA,3173994.0,25029170.0,258.07999,-0.034962
3785682,93436,2023-09-29,88160R10,11.0,3.0,TSLA,3179000.0,24395440.0,250.22000,-0.030456
3785683,93436,2023-10-31,88160R10,11.0,3.0,TSLA,3178921.0,25905681.0,200.84000,-0.197346
3785684,93436,2023-11-30,88160R10,11.0,3.0,TSLA,3178921.0,26395792.0,240.08000,0.195379


In [10]:
crsp.df.to_parquet('data/crsp_monthly_clean.parquet')